In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import os
import time
import sklearn
import numpy as np
import pandas as pd
from tqdm import tqdm
from random import random
from tabulate import tabulate

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.tree import DecisionTreeClassifier
evaluate={'Acc':"Accuracy", 'b_Acc':"Balanced Accuracy", 'F1':"F1 Score", 'kap':"Kappa", 'ROC':"Roc"}

In [3]:
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score


In [4]:
def folder(f_name): #this function creates a folder named "attacks" in the program directory.
    try:
        if not os.path.exists(f_name):
            os.makedirs(f_name)
    except OSError:
        print ("The folder could not be created!")

In [5]:
def find_the_way(path,file_format,con=""):
    files_add = []
    # r=root, d=directories, f = files
    for r, d, f in os.walk(path):
        for file in f:
            if file_format in file:
                if con in file:
                    files_add.append(os.path.join(r, file))  
            
    return files_add
path="./csvs"
csv_list=find_the_way(path,".csv")
csv_list

['./csvs\\ARP.csv',
 './csvs\\DHCP.csv',
 './csvs\\DHCPV6.csv',
 './csvs\\DHCP_1.csv',
 './csvs\\DHCP_2.csv',
 './csvs\\DHCP_3.csv',
 './csvs\\DHCP_4.csv',
 './csvs\\DHCP_5.csv',
 './csvs\\DNS.csv',
 './csvs\\DNS_1.csv',
 './csvs\\DNS_2.csv',
 './csvs\\EAPOL.csv',
 './csvs\\Ethernet.csv',
 './csvs\\HTTP.csv',
 './csvs\\HTTP2.csv',
 './csvs\\HTTP3.csv',
 './csvs\\ICMP.csv',
 './csvs\\ICMP6.csv',
 './csvs\\ICMP6_1.csv',
 './csvs\\ICMP6_2.csv',
 './csvs\\ICMP6_3.csv',
 './csvs\\ICMP6_4.csv',
 './csvs\\IGMP.csv',
 './csvs\\IPv4.csv',
 './csvs\\IPv6.csv',
 './csvs\\IPv6_1.csv',
 './csvs\\IPv6_2.csv',
 './csvs\\NTP.csv',
 './csvs\\NTP1.csv',
 './csvs\\NTP2.csv',
 './csvs\\QUIC.csv',
 './csvs\\STP.csv',
 './csvs\\STUN.csv',
 './csvs\\TCP.csv',
 './csvs\\TLS.csv',
 './csvs\\TLS_1.csv',
 './csvs\\TLS_2.csv',
 './csvs\\TLS_3.csv',
 './csvs\\UDP.csv',
 './csvs\\Wireshark.csv']

# SPLIT DATA as TRAIN AND TEST

In [6]:
ml_list={"DT":DecisionTreeClassifier()}#,"SVC":SVC()}}

In [7]:
def target_name(name):
    df = pd.read_csv(name)
    target_names=sorted(list(df[df.columns[-1]].unique()))
    return target_names

In [8]:
folder("results")

## Calculation of evaluations

In [9]:
def score(train_time,test_time,predict,y_test,class_based_results,repeat,cv,dname,ml,sw):
    train_time=train_time[0]
    test_time=test_time[0]
    rc=sklearn.metrics.recall_score(y_test, predict,average= "macro")
    pr=sklearn.metrics.precision_score(y_test, predict,average= "macro")
    f_1=sklearn.metrics.f1_score(y_test, predict,average= "macro")     
    accuracy=sklearn.metrics.accuracy_score(y_test, predict)
    accuracy_b=sklearn.metrics.balanced_accuracy_score( y_test,predict)
    kappa=sklearn.metrics.cohen_kappa_score(y_test, predict,labels=None, weights=None, sample_weight=None)
    try:
        roc=sklearn.metrics.roc_auc_score(y_test, predict)
    except:roc=np.nan
    report = sklearn.metrics.classification_report(y_test, predict, target_names=target_names,output_dict=True)
    cr = pd.DataFrame(report).transpose()
    line=[dname,sw,repeat,cv,ml,accuracy,accuracy_b,pr,rc,f_1,kappa,roc,train_time,test_time]

    if class_based_results.empty:
        class_based_results =cr
    else:
        class_based_results = class_based_results.add(cr, fill_value=0)
    return class_based_results,line

In [10]:
def ML_CV(loop1,output_csv,cols,dname):
    fold=2
    repetition=2
    sw=np.nan
    for ii in ml_list:
        class_based_results=pd.DataFrame()#"" #pd.DataFrame(0, index=np.arange((len(target_names)+3)), columns=["f1-score","precision","recall","support"])
        cm=pd.DataFrame()
        cv=0
        lines=[["Dataset","SW","T","CV","ML","Acc","b_Acc","Prec","Rec","F1","kap","ROC","tra-T","test-T"]]
        cm = pd.DataFrame()
        for i in range(repetition):

            rnd = random()
            
            kfold = sklearn.model_selection.KFold(n_splits=fold, shuffle=True, random_state=int(rnd*100))  
            cv=0
            df = pd.read_csv(loop1,usecols=cols)#,index_col=0)#,header=None )
            #df = df.reset_index(drop=True)
            df=df.fillna(0)

            #del df["MAC"] # if dataset has MAC colomn please uncomment this line
            X =df[df.columns[0:-1]]
            X=np.array(X)
            df[df.columns[-1]] = df[df.columns[-1]].astype('category')
            y=df[df.columns[-1]].cat.codes  
            X.shape
            for train_index, test_index in kfold.split(X):


                X_train, X_test = X[train_index], X[test_index]
                y_train, y_test = y[train_index], y[test_index]  

      


                #dname=loop1  [6:-13]  
                results_y=[]
                cv+=1
                results_y.append(y_test)


                precision=[]
                recall=[]
                f1=[]
                accuracy=[]
                train_time=[]
                test_time=[]
                total_time=[]
                kappa=[]
                accuracy_b=[]

                    #machine learning algorithm is applied in this section
                clf = ml_list[ii]#choose algorithm from ml_list dictionary
                second=time.time()
                clf.fit(X_train, y_train)
                train_time.append(float((time.time()-second)) )
                second=time.time()
                predict =clf.predict(X_test)
                test_time.append(float((time.time()-second)) )
    
                altime=0
                class_based_results,line=score(train_time,test_time,predict,y_test,class_based_results,cv,i,dname,ii,sw)
                lines.append(line)

                
                
                
                
                df_cm = pd.DataFrame(sklearn.metrics.confusion_matrix(y_test, predict))
                if cm.empty:
                    cm =df_cm
                else:
                    cm = cm.add(df_cm, fill_value=0)
                
                
                
                
                
        results = pd.DataFrame (lines[1:], columns = lines[0])
        results.to_csv(output_csv.replace("ML",ii),index=False)
        results=results.round(3)
        del results["SW"]
        del results["ROC"]
        #print (tabulate(results, headers=list(results.columns),tablefmt="double_grid"))
        #print()
        if False:
            cm=cm//(repetition*cv)
            graph_name=output_csv[:-4]+"_confusion matrix.pdf"   
            plt.figure(figsize = (10,7))
            sns.heatmap(cm,xticklabels=target_names, yticklabels=target_names, annot=True, fmt='g')
            plt.savefig(graph_name,bbox_inches='tight')#, dpi=400)
            plt.show()
            #print(cm)
            print("\n\n\n")             




# 10 fold 10 times Cross-Validating

In [11]:
features=['dns.time', 'http.time', 'icmp.data_time_relative', 'icmp.resptime', 'stun.time', 'tcp.ack',
          'tcp.ack_raw', 'tcp.analysis.ack_rtt', 'tcp.analysis.initial_rtt', 'tcp.analysis.rto', 'tcp.nxtseq', 
          'tcp.options.timestamp.tsecr', 'tcp.options.timestamp.tsval', 'tcp.seq', 'tcp.seq_raw', 'tcp.time_delta',
          'tcp.time_relative', 'udp.time_delta', 'udp.time_relative', 'arp.hw.size', 'arp.hw.type', 'arp.isannouncement',
          'arp.isgratuitous', 'arp.isprobe', 'arp.opcode', 'arp.proto.size', 'arp.seconds-since-duplicate-address-frame', 'dhcp.flags.bc',
          'dhcp.hops', 'dhcp.hw.addr_padding', 'dhcp.hw.len', 'dhcp.option.dhcp', 'dhcp.option.dhcp_max_message_size', 'dhcp.option.end', 
          'dhcp.option.interface_mtu', 'dhcp.option.ip_address_lease_time', 'dhcp.option.padding', 'dhcp.option.rebinding_time_value',
          'dhcp.option.renewal_time_value', 'dhcp.secs', 'dhcp.type', 'dns.count.add_rr', 'dns.count.answers', 'dns.count.auth_rr', 
          'dns.count.queries', 'dns.extraneous', 'dns.extraneous.data', 'dns.extraneous.length', 'dns.flags.authenticated', 
          'dns.flags.authoritative', 'dns.flags.checkdisable', 'dns.flags.opcode', 'dns.flags.rcode', 'dns.flags.recavail',
          'dns.flags.recdesired', 'dns.flags.response', 'dns.flags.truncated', 'dns.flags.z', 'dns.response_to', 'dns.retransmission',
          'dns.retransmit_request', 'dns.retransmit_request_in', 'dns.retransmit_response', 'dns.retransmit_response_in', 
          'dns.soa.expire_limit', 'dns.soa.minimum_ttl', 'dns.soa.refresh_interval', 'dns.soa.retry_interval', 'dns.soa.serial_number',
          'dns.unsolicited', 'eapol.keydes.key_iv', 'eapol.keydes.key_len', 'eapol.keydes.replay_counter', 'eapol.keydes.type', 
          'eapol.len', 'eapol.type', 'eapol.version', 'eth.dst.ig', 'eth.dst.lg', 'eth.dst.oui', 'eth.fcs.status', 'eth.len', 'eth.padding',
          'eth.padding_bad', 'eth.src.ig', 'eth.src.lg', 'eth.src.oui', 'http.chat', 'http.content_length', 'http.content_length_header',
          'http.notification', 'http.prev_request_in', 'http.prev_response_in', 'http.request', 'http.request_in', 'http.request_number',
          'http.response', 'http.response.code', 'http.response_number', 'icmp.checksum.status', 'icmp.code', 'icmp.ident', 'icmp.ident_le', 'icmp.resp_to', 'icmp.seq', 'icmp.seq_le', 'icmp.type', 'icmp.unused', 'icmpv6.checksum.status', 'icmpv6.code', 'icmpv6.mldr.nb_mcast_records', 'icmpv6.opt', 'icmpv6.opt.length', 'icmpv6.opt.type', 'icmpv6.reserved', 'icmpv6.type', 'igmp.checksum.status', 'igmp.max_resp', 'igmp.num_grp_recs', 'igmp.version', 'ip.bogus_ip_length', 'ip.len', 'ip.opt.len', 'ip.opt.ra', 'ip.opt.type', 'ip.opt.type.class', 'ip.opt.type.copy', 'ip.opt.type.number', 'ipv6.hlim', 'ipv6.hopopts.nxt', 'ipv6.nxt', 'ipv6.opt.router_alert', 'ipv6.plen', 'ipv6.tclass.dscp', 'ipv6.tclass.ecn', 'ipv6.version', 'ntp.flags.li', 'ntp.flags.mode', 'ntp.flags.vn', 'ntp.ppoll', 'ntp.precision', 'ntp.rootdelay', 'ntp.rootdispersion', 'ntp.stratum', 'stun.att.crc32.status', 'stun.att.error', 'stun.att.error.class', 'stun.att.lifetime', 'stun.attributes', 'stun.length', 'stun.network_version', 'stun.response-to', 'tcp.analysis', 'tcp.analysis.ack_lost_segment', 'tcp.analysis.acks_frame', 'tcp.analysis.bytes_in_flight', 'tcp.analysis.duplicate_ack_frame', 'tcp.analysis.duplicate_ack_num', 'tcp.analysis.flags', 'tcp.analysis.keep_alive', 'tcp.analysis.keep_alive_ack', 'tcp.analysis.lost_segment', 'tcp.analysis.out_of_order', 'tcp.analysis.push_bytes_sent', 'tcp.analysis.retransmission', 'tcp.analysis.reused_ports', 'tcp.analysis.rto_frame', 'tcp.analysis.spurious_retransmission', 'tcp.analysis.window_update', 'tcp.checksum.status', 'tcp.completeness', 'tcp.connection.fin', 'tcp.connection.fin_active', 'tcp.connection.fin_passive', 'tcp.connection.rst', 'tcp.connection.syn', 'tcp.connection.synack', 'tcp.dstport', 'tcp.fin_retransmission', 'tcp.flags.ack', 'tcp.flags.ae', 'tcp.flags.cwr', 'tcp.flags.ece', 'tcp.flags.fin', 'tcp.flags.push', 'tcp.flags.res', 'tcp.flags.reset', 'tcp.flags.syn', 'tcp.flags.urg', 'tcp.hdr_len', 'tcp.len', 'tcp.options.mss_val', 'tcp.options.sack.count', 'tcp.options.sack.dsack', 'tcp.options.sack.dsack_le', 'tcp.options.sack.dsack_re', 'tcp.options.sack_le', 'tcp.options.sack_perm', 'tcp.options.sack_re', 'tcp.options.wscale.multiplier', 'tcp.options.wscale.shift', 'tcp.pdu.size', 'tcp.reassembled.length', 'tcp.segment.count', 'tcp.segment.overlap', 'tcp.segments', 'tcp.srcport', 'tcp.stream', 'tcp.urgent_pointer', 'tcp.window_size', 'tcp.window_size_scalefactor', 'tcp.window_size_value', 'tls.alert_message', 'tls.change_cipher_spec', 'tls.handshake.certificates', 'tls.handshake.certificates_length', 'tls.handshake.challenge', 'tls.handshake.challenge_length', 'tls.handshake.cipher_spec_len', 'tls.handshake.cipher_suites_length', 'tls.handshake.ciphersuites', 'tls.handshake.client_point_len', 'tls.handshake.comp_method', 'tls.handshake.comp_methods', 'tls.handshake.comp_methods_length', 'tls.handshake.epms_len', 'tls.handshake.extension.heartbeat.mode', 'tls.handshake.extensions_ec_point_formats', 'tls.handshake.extensions_ec_point_formats_length', 'tls.handshake.extensions_length', 'tls.handshake.extensions_reneg_info_len', 'tls.handshake.extensions_server_name_len', 'tls.handshake.extensions_server_name_list_len', 'tls.handshake.extensions_server_name_type', 'tls.handshake.extensions_supported_groups', 'tls.handshake.extensions_supported_groups_length', 'tls.handshake.g_len', 'tls.handshake.p_len', 'tls.handshake.server_point_len', 'tls.handshake.session_id_length', 'tls.handshake.session_ticket_length', 'tls.handshake.session_ticket_lifetime_hint', 'tls.handshake.sig_hash_alg_len', 'tls.handshake.sig_hash_algs', 'tls.handshake.sig_len', 'tls.handshake.yc_len', 'tls.handshake.ys_len', 'tls.resumed', 'tls.sct.scts_length', 'udp.checksum.status', 'udp.dstport', 'udp.length', 'udp.length.bad', 'udp.possible_traceroute', 'udp.srcport', 'udp.stream', 'eth.dst', 'eth.src', 'ipv6.dst', 'ipv6.src', '_ws.col.Destination', '_ws.col.Source', 
          'stun.cookie', 'stun.id', 'stun.value', 'tcp.options', 'tcp.options.mss', 'tcp.options.sack', 'tcp.options.wscale', 'ntp.refid', 'stun.att.username', 'arp.proto.type', 'dhcp.flags', 'dhcp.flags.reserved', 'dhcp.id', 'dns.flags', 'dns.id', 'eth.fcs', 'eth.type', 'icmp.checksum', 'icmpv6.checksum', 'igmp.checksum', 'igmp.type', 'ip.id', 'ipv6.flow', 'ipv6.tclass', 'ntp.flags', 'stun.att.crc32', 'stun.att.transp', 'stun.type', 'stun.type.class', 'stun.type.method', 'stun.type.method-assignment', 'tcp.checksum', 'tcp.flags', 'tls.handshake.server_curve_type', 'tls.handshake.server_named_curve', 'tls.handshake.version', 'udp.checksum', 'http.connection', 'http.request.method', 'http.request.uri.path', 'http.response.code.desc', 'http.response.phrase', 'dns.srv.proto', 'tcp.flags.str', '_ws.col.Protocol', 'tls.handshake.sig_hash_alg', 'dhcp.hw.type', 'ip.dsfield', 'ip.flags', 'ipv6.opt.type', 'ipv6.opt.type.rest', 'stun.att.family', 'stun.att.ipv4-xord', 'stun.att.port-xord', 'tcp.segment', 'arp.duplicate-address-detected', 'arp.duplicate-address-frame', 'dns.srv.port', 'dns.srv.priority', 'dns.srv.weight', 'eth.addr.oui', 'eth.ig', 'eth.lg', 'ip.checksum.status', 'ip.dsfield.dscp', 'ip.dsfield.ecn', 'ip.flags.df', 'ip.flags.mf', 'ip.flags.rb', 'ip.frag_offset', 'ip.hdr_len', 'ip.proto', 'ip.ttl', 'ip.version', 'ipv6.opt', 'ipv6.opt.length', 'ipv6.opt.type.action', 'ipv6.opt.type.change', 'stun.att.padding', 'stun.att.port', 'stun.att.reserved', 'tcp.analysis.duplicate_ack', 'tcp.port', 'tls.record.length', 'udp.port', 'ip.dst', 'ip.src', 'ip.src_host',
          'stun.att.ipv4', 'dhcp.hw.mac_addr', 'Payload_Entropy', 'dstport', 'srcport', 'dstport_class', 'srcport_class', 'Label']




In [12]:
train="./ML.csv"
output_csv="./results/CV.csv"
target_names=[0,1]
dname="IRIS_10X10_CV"

In [13]:
errors=[]
for i in tqdm(features):
    feature=[i,"Label"]
    try:
        ML_CV(train,output_csv,feature,i)  
    except:
        try:
            print(i)
            df = pd.read_csv(train,usecols=feature)
            print(df.groupby(i).size())
            errors.append(i)
        except:
            print(f"errror ",feature)

  7%|█████▋                                                                         | 26/358 [05:15<1:01:32, 11.12s/it]

arp.seconds-since-duplicate-address-frame


  8%|██████                                                                           | 27/358 [05:24<58:12, 10.55s/it]

arp.seconds-since-duplicate-address-frame
-9999    191159
302.0         1
303.0         1
304.0         1
309.0         1
          ...  
95737         1
95921         1
95923         1
96126         1
96128         1
Length: 1680, dtype: int64


 34%|███████████████████████████                                                     | 121/358 [22:55<43:55, 11.12s/it]

...

dhcp.hw.mac_addr
-9999                 38912
-9999                156797
00:17:88:24:76:ff       120
00:1a:22:03:cb:be        32
00:24:e4:24:80:2a       100
1c:5f:2b:aa:fd:4e       128
20:f8:5e:ca:91:52       158
38:0b:40:ef:85:41        10
3c:49:37:03:17:db        46
3c:49:37:03:17:f0        36
50:c7:bf:00:c7:03        86
50:c7:bf:00:fc:a3        82
5c:cf:7f:06:d9:02        80
5c:cf:7f:07:ae:fb        82
6c:72:20:c5:17:5a        48
74:da:38:23:22:7b        55
74:da:38:4a:76:49        55
74:da:38:80:79:fc        22
74:da:38:80:7a:08        24
84:18:26:7b:5f:6b       156
90:8d:78:a8:e1:43        80
90:8d:78:a9:3d:6f        43
90:8d:78:dd:0d:60        47
94:10:3e:34:0c:b5        50
94:10:3e:35:01:c1        80
94:10:3e:41:c2:05       144
94:10:3e:42:80:69        40
94:10:3e:cd:37:65        80
ac:cf:23:62:3c:6e        80
b0:c5:54:1c:71:85        40
b0:c5:54:25:5b:0e        50
dtype: int64


100%|█████████████████████████████████████████████████████████████████████████████▊| 357/358 [1:07:00<00:11, 11.18s/it]

Label


100%|██████████████████████████████████████████████████████████████████████████████| 358/358 [1:07:09<00:00, 11.26s/it]

Label
0    98745
1    99018
dtype: int64


In [14]:
errors

['arp.seconds-since-duplicate-address-frame',
 'ip.len',
 'eth.dst',
 'eth.src',
 'ipv6.dst',
 'ipv6.src',
 '_ws.col.Destination',
 '_ws.col.Source',
 'tcp.options',
 'ip.id',
 'ip.dst',
 'ip.src',
 'ip.src_host',
 'stun.att.ipv4',
 'dhcp.hw.mac_addr',
 'Label']

In [ ]:
['arp.seconds-since-duplicate-address-frame',
 'ip.len',
# eth.dst',
#'eth.src',
# 'ipv6.dst',
# 'ipv6.src',
# '_ws.col.Destination',
# '_ws.col.Source',
######### 'http.user_agent',
 'stun.cookie',
 'stun.id',
 'stun.value',
 'tcp.options',
 'tcp.options.mss',
 'tcp.options.sack',
 'tcp.options.wscale',
 'ntp.refid',
 'stun.att.username',
 'arp.proto.type',
 'dhcp.flags',
 'dhcp.flags.reserved',
 'dhcp.id',
 'dns.flags',
 'dns.id',
 'eth.fcs',
 'eth.type',
 'icmp.checksum',
 'icmpv6.checksum',
 'igmp.checksum',
 'igmp.type',
 'ip.id',
 'ipv6.flow',
 'ipv6.tclass',
 'ntp.flags',
 'stun.att.crc32',
 'stun.att.transp',
 'stun.type',
 'stun.type.class',
 'stun.type.method',
 'stun.type.method-assignment',
 'tcp.checksum',
 'tcp.flags',
 'tls.handshake.server_curve_type',
 'tls.handshake.server_named_curve',
 'tls.handshake.version',
 'udp.checksum',
 '_ws.col.Protocol',
 'tls.handshake.sig_hash_alg',
 'dhcp.hw.type',
 'ip.dsfield',
 'ip.flags',
 'ipv6.opt.type',
 'ipv6.opt.type.rest',
 'stun.att.family',
 'stun.att.ipv4-xord',
 'stun.att.port-xord',
 #'ip.dst',
 #'ip.src',
 #'ip.src_host',
 #'stun.att.ipv4',
 #'dhcp.hw.mac_addr'

]

In [ ]:
features=['arp.seconds-since-duplicate-address-frame',
 'ip.len',
# eth.dst',
#'eth.src',
# 'ipv6.dst',
# 'ipv6.src',
# '_ws.col.Destination',
# '_ws.col.Source',
######### 'http.user_agent',
 'stun.cookie',
 'stun.id',
 'stun.value',
 'tcp.options',
 'tcp.options.mss',
 'tcp.options.sack',
 'tcp.options.wscale',
 'ntp.refid',
 'stun.att.username',
 'arp.proto.type',
 'dhcp.flags',
 'dhcp.flags.reserved',
 'dhcp.id',
 'dns.flags',
 'dns.id',
 'eth.fcs',
 'eth.type',
 'icmp.checksum',
 'icmpv6.checksum',
 'igmp.checksum',
 'igmp.type',
 'ip.id',
 'ipv6.flow',
 'ipv6.tclass',
 'ntp.flags',
 'stun.att.crc32',
 'stun.att.transp',
 'stun.type',
 'stun.type.class',
 'stun.type.method',
 'stun.type.method-assignment',
 'tcp.checksum',
 'tcp.flags',
 'tls.handshake.server_curve_type',
 'tls.handshake.server_named_curve',
 'tls.handshake.version',
 'udp.checksum',
 '_ws.col.Protocol',
 'tls.handshake.sig_hash_alg',
 'dhcp.hw.type',
 'ip.dsfield',
 'ip.flags',
 'ipv6.opt.type',
 'ipv6.opt.type.rest',
 'stun.att.family',
 'stun.att.ipv4-xord',
 'stun.att.port-xord'
 #'ip.dst',
 #'ip.src',
 #'ip.src_host',
 #'stun.att.ipv4',
 #'dhcp.hw.mac_addr',
]

In [41]:
errors

['arp.seconds-since-duplicate-address-frame',
 'ip.len',
 'eth.dst',
 'eth.src',
 'ipv6.dst',
 'ipv6.src',
 '_ws.col.Destination',
 '_ws.col.Source',
 'http.user_agent',
 'stun.cookie',
 'stun.id',
 'stun.value',
 'tcp.options',
 'tcp.options.mss',
 'tcp.options.sack',
 'tcp.options.wscale',
 'ntp.refid',
 'stun.att.username',
 'arp.proto.type',
 'dhcp.flags',
 'dhcp.flags.reserved',
 'dhcp.id',
 'dns.flags',
 'dns.id',
 'eth.fcs',
 'eth.type',
 'icmp.checksum',
 'icmpv6.checksum',
 'igmp.checksum',
 'igmp.type',
 'ip.id',
 'ipv6.flow',
 'ipv6.tclass',
 'ntp.flags',
 'stun.att.crc32',
 'stun.att.transp',
 'stun.type',
 'stun.type.class',
 'stun.type.method',
 'stun.type.method-assignment',
 'tcp.checksum',
 'tcp.flags',
 'tls.handshake.server_curve_type',
 'tls.handshake.server_named_curve',
 'tls.handshake.version',
 'udp.checksum',
 '_ws.col.Protocol',
 'tls.handshake.sig_hash_alg',
 'dhcp.hw.type',
 'ip.dsfield',
 'ip.flags',
 'ipv6.opt.type',
 'ipv6.opt.type.rest',
 'stun.att.famil

In [18]:
train="./selected.csv"
df = pd.read_csv(train,usecols=features)
df


,arp.seconds-since-duplicate-address-frame,ip.len,stun.cookie,stun.id,stun.value,tcp.options,tcp.options.mss,tcp.options.sack,tcp.options.wscale,ntp.refid,...,_ws.col.Protocol,tls.handshake.sig_hash_alg,dhcp.hw.type,ip.dsfield,ip.flags,ipv6.opt.type,ipv6.opt.type.rest,stun.att.family,stun.att.ipv4-xord,stun.att.port-xord
0,NaN,NaN,[nan nan nan ... nan nan nan],[nan nan nan ... nan nan nan],[nan nan nan ... nan nan nan],[nan nan nan ... nan nan nan],[nan nan nan ... nan nan nan],[nan nan nan ... nan nan nan],[nan nan nan ... nan nan nan],[nan nan nan ... nan nan nan],...,1,[nan nan nan ... nan nan nan],[nan nan nan ... nan nan nan],[nan nan nan ... '0x00' '0x00' '0x00'],[nan nan nan ... '0x00' '0x02' '0x00'],[nan nan nan ... nan nan nan],[nan nan nan ... nan nan nan],[nan nan nan ... nan nan nan],[nan nan nan ... nan nan nan],[nan nan nan ... nan nan nan]
1,NaN,NaN,[nan nan nan ... nan nan nan],[nan nan nan ... nan nan nan],[nan nan nan ... nan nan nan],[nan nan nan ... nan nan nan],[nan nan nan ... nan nan nan],[nan nan nan ... nan nan nan],[nan nan nan ... nan nan nan],[nan nan nan ... nan nan nan],...,1,[nan nan nan ... nan nan nan],[nan nan nan ... nan nan nan],[nan nan nan ... '0x00' '0x00' '0x00'],[nan nan nan ... '0x00' '0x02' '0x00'],[nan nan nan ... nan nan nan],[nan nan nan ... nan nan nan],[nan nan nan ... nan nan nan],[nan nan nan ... nan nan nan],[nan nan nan ... nan nan nan]
2,NaN,NaN,[nan nan nan ... nan nan nan],[nan nan nan ... nan nan nan],[nan nan nan ... nan nan nan],[nan nan nan ... nan nan nan],[nan nan nan ... nan nan nan],[nan nan nan ... nan nan nan],[nan nan nan ... nan nan nan],[nan nan nan ... nan nan nan],...,1,[nan nan nan ... nan nan nan],[nan nan nan ... nan nan nan],[nan nan nan ... '0x00' '0x00' '0x00'],[nan nan nan ... '0x00' '0x02' '0x00'],[nan nan nan ... nan nan nan],[nan nan nan ... nan nan nan],[nan nan nan ... nan nan nan],[nan nan nan ... nan nan nan],[nan nan nan ... nan nan nan]
3,NaN,NaN,[nan nan nan ... nan nan nan],[nan nan nan ... nan nan nan],[nan nan nan ... nan nan nan],[nan nan nan ... nan nan nan],[nan nan nan ... nan nan nan],[nan nan nan ... nan nan nan],[nan nan nan ... nan nan nan],[nan nan nan ... nan nan nan],...,1,[nan nan nan ... nan nan nan],[nan nan nan ... nan nan nan],[nan nan nan ... '0x00' '0x00' '0x00'],[nan nan nan ... '0x00' '0x02' '0x00'],[nan nan nan ... nan nan nan],[nan nan nan ... nan nan nan],[nan nan nan ... nan nan nan],[nan nan nan ... nan nan nan],[nan nan nan ... nan nan nan]
4,NaN,401,[nan nan nan ... nan nan nan],[nan nan nan ... nan nan nan],[nan nan nan ... nan nan nan],[nan nan nan ... nan nan nan],[nan nan nan ... nan nan nan],[nan nan nan ... nan nan nan],[nan nan nan ... nan nan nan],[nan nan nan ... nan nan nan],...,7,[nan nan nan ... nan nan nan],[nan nan nan ... nan nan nan],0,2,[nan nan nan ... nan nan nan],[nan nan nan ... nan nan nan],[nan nan nan ... nan nan nan],[nan nan nan ... nan nan nan],[nan nan nan ... nan nan nan]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197758,NaN,84.0,[nan nan nan ... nan nan nan],[nan nan nan ... nan nan nan],[nan nan nan ... nan nan nan],[nan nan nan ... nan nan nan],[nan nan nan ... nan nan nan],[nan nan nan ... nan nan nan],[nan nan nan ... nan nan nan],[nan nan nan ... nan nan nan],...,6,[nan nan nan ... nan nan nan],[nan nan nan ... nan nan nan],0,0,[nan nan nan ... nan nan nan],[nan nan nan ... nan nan nan],[nan nan nan ... nan nan nan],[nan nan nan ... nan nan nan],[nan nan nan ... nan nan nan]
197759,NaN,84.0,[nan nan nan ... nan nan nan],[nan nan nan ... nan nan nan],[nan nan nan ... nan nan nan],[nan nan nan ... nan nan nan],[nan nan nan ... nan nan nan],[nan nan nan ... nan nan nan],[nan nan nan ... nan nan nan],[nan nan nan ... nan nan nan],...,6,[nan nan nan ... nan nan nan],[nan nan nan ... nan nan nan],0,2,[nan nan nan ... nan nan nan],[nan nan nan ... nan nan nan],[nan nan nan ... nan nan nan],[nan nan nan ... nan nan nan],[nan nan nan ... nan 

In [15]:
train="./nihai.csv"

df = pd.read_csv(train,usecols=features,sep="|")
df

,arp.proto.type,arp.seconds-since-duplicate-address-frame,dhcp.flags,dhcp.flags.reserved,dhcp.hw.type,dhcp.id,dns.flags,dns.id,eth.fcs,eth.type,...,tcp.options,tcp.options.mss,tcp.options.sack,tcp.options.wscale,tls.handshake.server_curve_type,tls.handshake.server_named_curve,tls.handshake.sig_hash_alg,tls.handshake.version,udp.checksum,_ws.col.Protocol
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0x888e,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EAPOL
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0x888e,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EAPOL
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0x888e,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EAPOL
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0x888e,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EAPOL
4,NaN,NaN,NaN,NaN,NaN,NaN,0x8400,0x0000,NaN,0x0800,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0x5eab,MDNS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197758,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0x0800,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ICMP
197759,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0x0800,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ICMP
197760,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0x0800,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ICMP
197761,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0x0800,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ICMP
